##  sky画像の作成  
makedark.ipynbまでで作成した関数により、1Expあたりのダークまでは得られた

In [1]:
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil